# Limpieza: Predicción de la presencia altos niveles de PM10

En este notebook vamos a tomar los datasets de contaminantes y metereología obtenidos de http://www.aire.cdmx.gob.mx/default.php

Juntaremos los dataframes con una PivotTable y las agruparemos por el momento de la medición

In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
import pandas as pd
import matplotlib
import seaborn as sns

from datetime import timedelta

Definimos unas funciones para convertir las fechas de DD-MM-AAAA a AAAA-MM-DD y para convertir las horas 24:00 en 00:00:

In [2]:
def time_converter(x):
    x0 = x.split(" ")[0]
    x0 = x0.split("/")
    x1 = x.split(" ")[1]
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00"
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [3]:
def time_converter_guion(x):
    x0 = x.split(" ")[0]
    x0 = x0.split("-")
    x1 = x.split(" ")[1]
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00"
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [4]:
def time_converter_date(x):
    x0 = x.split("/")
    return x0[2]+"-"+x0[1]+"-"+x0[0]

## Carga de los conjuntos de datos   <a class="anchor" id="limpieza-bullet"></a>

Definamos el año a limpiar:

In [675]:
a="2013"

Cargamos los datos

### Presión

In [676]:
pre_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/presion/PA_' + a + ".csv"),header=8)

Eliminamos las entradas conn NAN y eliminamos la columna unit:

In [677]:
pre_2018 = pre_2018.dropna(how='any') 
pre_2018 = pre_2018.drop(['unit'], axis=1)

### Metereología

In [678]:
met_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Metereología/meteorología_' + a + ".CSV"),header=10)
met_2018 = met_2018.dropna(how='any')
met_2018 = met_2018.drop(['unit'], axis=1)

### Contaminantes

In [679]:
cont_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Contaminantes/contaminantes_'+ a +'.CSV'),header=10)
cont_2018 = cont_2018.dropna(how='any')
cont_2018 = cont_2018.drop(['unit'], axis=1)

### Radiación UVA

In [680]:
radA_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVA/UVA_'+ a +'.csv'),header=8)
radA_2018 = radA_2018.fillna(0)
radA_2018 = radA_2018.drop(['unit'], axis=1)

### Radiación UVB

In [681]:
radB_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVB/UVB_'+ a +'.csv'),header=8)
radB_2018 = radB_2018.fillna(0)
radB_2018 = radB_2018.drop(['unit'], axis=1)

## Append Radiación

Juntamos los df. de UVA y UVBm

In [682]:
rad_2018 = radA_2018.append(radB_2018, ignore_index=True)

## Precipitación

In [683]:
prec_2018 = pd.read_excel(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Precipitación/'+a+"PPH.xls"))

Los valores vacíos vienen como -99, vamos a eliminarlos:

In [684]:
prec_2018 = prec_2018.where(prec_2018.LOM != -99.00)

In [685]:
prec_2018 = prec_2018.dropna()

Renombramos algunas columnas:

In [686]:
prec_2018 = prec_2018.rename(columns={'FECHA': 'fecha'})

Transponemos el df:

In [687]:
prec_2018 = pd.melt(prec_2018, id_vars=["fecha"], 
                   var_name="id_station",value_name="Precip")

In [688]:
prec_2018['fecha'] =  pd.to_datetime(prec_2018['fecha'], format='%Y-%m-%d %H:%M')

# Limpieza paso a paso

### Pres

Nos quedamos con las columnas indicadas y reseteamos el index:

In [689]:
pre_ACO = pre_2018
pre_ACO = pre_ACO.reset_index(drop=False)
pre_ACO = pre_ACO[["Date","cve_station","parameter","value"]]

In [690]:
pre_ACO = pre_ACO.rename(columns={'Date': 'date', 'cve_station': 'id_station','parameter': 'id_parameter'})

### Contaminantes

In [691]:
cont_ACO = cont_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
cont_ACO = cont_ACO.reset_index(drop=False)
cont_ACO = cont_ACO[["date","id_station","id_parameter","value"]]

### Metereologia

In [692]:
met_ACO = met_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
met_ACO = met_ACO.reset_index(drop=False)
met_ACO = met_ACO[["date","id_station","id_parameter","value"]]

### Radiación

In [693]:
rad_2018 = rad_2018.rename(columns={'Date': 'date', 'cve_station': 'id_station','parameter': 'id_parameter'})

In [694]:
rad_ACO = rad_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO = rad_ACO[["date","id_station","id_parameter","value"]]

## Pivot_Table <a class="anchor" id="pivot-bullet"></a>

### Pre

Creamos la tabla dinámica o pivote:

In [695]:
pre_ACO_hour = pd.pivot_table(pre_ACO,index=["date","id_station"],columns=["id_parameter"])

Reseteamos el index para desacernos del multiindex:

In [696]:
pre_ACO_hour = pre_ACO_hour.reset_index(drop=False)

In [697]:
pre_ACO_hour.columns = pre_ACO_hour.columns.droplevel()

Reefinimos las columnas:

In [698]:
pre_ACO_hour["id_station"] = pre_ACO_hour.iloc[:,1]
pre_ACO_hour["date"] = pre_ACO_hour.iloc[:,0]

Nos deshacemos de las columnas sin nombre:

In [699]:
pre_ACO_hour = pre_ACO_hour.drop([""],axis=1)

Aplicamos las funciones definidas al principio del documento:

In [700]:
pre_ACO_hour['date'] = pre_ACO_hour.apply(lambda row: time_converter_guion(row['date']), axis=1) 

Convertimos lacolumna fecha en datetime:

In [701]:
pre_ACO_hour['date'] =  pd.to_datetime(pre_ACO_hour['date'], format='%Y-%m-%d %H:%M')

In [702]:
pre_ACO_hour = pre_ACO_hour[[ "date" ,"id_station",'PA']]

### Contaminantes

In [703]:
cont_ACO_hour = pd.pivot_table(cont_ACO,index=["date","id_station"],columns=["id_parameter"])
cont_ACO_hour = cont_ACO_hour.reset_index(drop=False)
cont_ACO_hour.columns = cont_ACO_hour.columns.droplevel()
cont_ACO_hour["id_station"] = cont_ACO_hour.iloc[:,1]
cont_ACO_hour["date"] = cont_ACO_hour.iloc[:,0]
cont_ACO_hour = cont_ACO_hour.drop([""],axis=1)
cont_ACO_hour['date'] = cont_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
cont_ACO_hour['date'] =  pd.to_datetime(cont_ACO_hour['date'], format='%Y-%m-%d %H:%M')
cont_ACO_hour = cont_ACO_hour[[ "date" ,"id_station",'CO', 'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'PMCO', 'SO2','PM10']]

### Metereología

In [704]:
met_ACO_hour = pd.pivot_table(met_ACO,index=["date","id_station"],columns=["id_parameter"])
met_ACO_hour = met_ACO_hour.reset_index(drop=False)
met_ACO_hour.columns = met_ACO_hour.columns.droplevel()
met_ACO_hour["id_station"] = met_ACO_hour.iloc[:,1]
met_ACO_hour["date"] = met_ACO_hour.iloc[:,0]
met_ACO_hour = met_ACO_hour.drop([""],axis=1)
met_ACO_hour['date'] = met_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
met_ACO_hour['date'] =  pd.to_datetime(met_ACO_hour['date'], format='%Y-%m-%d %H:%M')
met_ACO_hour = met_ACO_hour[["date","id_station","RH","TMP","WSP","WDR"]]

### Radiación

In [705]:
rad_ACO = pd.pivot_table(rad_ACO,index=["date","id_station"],columns=["id_parameter"])
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO.columns = rad_ACO.columns.droplevel()
rad_ACO["id_station"] = rad_ACO.iloc[:,1]
rad_ACO["date"] = rad_ACO.iloc[:,0]
rad_ACO = rad_ACO.drop([""],axis=1)
rad_ACO['date'] = rad_ACO.apply(lambda row: time_converter_guion(row['date']), axis=1)
rad_ACO['date'] =  pd.to_datetime(rad_ACO['date'], format='%Y-%m-%d %H:%M')
rad_ACO = rad_ACO[[ "date","id_station",'UVA',"UVB"]]
rad_ACO = rad_ACO.dropna(how='any')

# Merge de Dataframes   <a class="anchor" id="merge-bullet"></a>

Por hora: no es posible añadir precipitación ya que de origen viene por día.

Juntamos los dataframes:

In [706]:
data_hour_merge = pd.merge(cont_ACO_hour, met_ACO_hour, on=["date","id_station"])
data_hour_merge = pd.merge(data_hour_merge, pre_ACO_hour, on=["date","id_station"])
data_hour_merge = pd.merge(data_hour_merge, rad_ACO, on=["date","id_station"])

Eliminamos los NAN:

In [707]:
data_hour_merge = data_hour_merge.dropna(how='any')

Definimos las columnas hora, dia, mes

In [708]:
data_hour_merge["hora"] = pd.DatetimeIndex(data_hour_merge['date']).hour
data_hour_merge["dia"] = pd.DatetimeIndex(data_hour_merge['date']).day
data_hour_merge["mes"]= pd.DatetimeIndex(data_hour_merge['date']).month


Cambiamos el nombre de date a fecha:

In [709]:
data_hour_merge = data_hour_merge.rename(columns={'date': 'fecha'})

In [710]:
data_hour_merge = data_hour_merge[["fecha",'hora',"dia", 'mes', 'id_station','UVA',"UVB", "PA",'CO', 'NO', 'NO2', 'NOX', 'O3',
       'PM2.5', 'PMCO', 'SO2', 'RH', 'TMP', 'WSP', 'WDR', 'PM10']]

Exportamos:

In [711]:
data_hour_merge.head()

id_parameter,fecha,hora,dia,mes,id_station,UVA,UVB,PA,CO,NO,...,NOX,O3,PM2.5,PMCO,SO2,RH,TMP,WSP,WDR,PM10
2,2013-01-01 01:00:00,1,1,1,TLA,0.0,0.0,583.0,1.2,38.0,...,69.0,1.0,48.0,21.0,3.0,59.0,13.2,0.8,235.0,69.0
5,2013-01-01 02:00:00,2,1,1,TLA,0.0,0.0,583.0,1.4,46.0,...,77.0,2.0,98.0,32.0,6.0,61.0,12.7,0.7,166.0,131.0
8,2013-01-01 03:00:00,3,1,1,TLA,0.0,0.0,582.0,1.8,73.0,...,99.0,2.0,109.0,34.0,7.0,64.0,11.9,0.6,218.0,142.0
11,2013-01-01 04:00:00,4,1,1,TLA,0.0,0.0,582.0,2.0,86.0,...,113.0,2.0,101.0,36.0,5.0,65.0,11.6,1.0,203.0,137.0
14,2013-01-01 05:00:00,5,1,1,TLA,0.0,0.0,582.0,2.2,102.0,...,129.0,2.0,90.0,30.0,5.0,68.0,10.8,0.4,261.0,120.0


In [654]:
data_hour_merge.to_csv(str("/Users/danielbustillos/Documents/servicio/Contaminación PM10/Outputs/por_hora/cont_hora" + a + ".csv"))

## Promedio Por día

In [713]:
data_day_avg = data_hour_merge

In [714]:
data_day_avg['fecha'] = data_hour_merge['fecha'].astype(str).str[0:10]

In [715]:
data_day_avg = data_day_avg.groupby(['fecha'])[ 'UVA', 'UVB', 'PA', 'CO',
       'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'PMCO', 'SO2', 'RH', 'TMP', 'WSP',
       'WDR', 'PM10'].mean()

In [716]:
data_day_avg = data_day_avg.reset_index(drop=False)

In [717]:
data_day_avg['fecha'] =  pd.to_datetime(data_day_avg['fecha'], format='%Y-%m-%d %H:%M')

In [718]:
prec_2018 = prec_2018.drop(["id_station"],axis=1)

In [719]:
data_day_avg = pd.merge(data_day_avg, prec_2018, on=["fecha"],  how='left')

In [720]:
data_day_avg = data_day_avg.fillna(0)

In [721]:
data_day_avg.head()

,fecha,UVA,UVB,PA,CO,NO,NO2,NOX,O3,PM2.5,PMCO,SO2,RH,TMP,WSP,WDR,PM10,Precip
0,2013-01-01,1.004579,0.630158,583.052632,1.118421,34.710526,28.763158,63.500000,21.657895,58.842105,26.368421,4.815789,50.289474,15.192105,1.707895,171.026316,85.236842,0.0
1,2013-01-02,0.533762,0.357000,582.976190,0.759524,29.000000,27.357143,56.476190,9.119048,22.595238,22.833333,11.547619,59.976190,13.826190,2.414286,197.166667,45.404762,0.0
2,2013-01-03,0.896400,0.588667,583.766667,0.926667,47.000000,32.366667,79.533333,12.166667,28.100000,28.633333,29.300000,55.033333,14.473333,2.500000,233.766667,56.800000,0.0
3,2013-01-04,0.362179,0.224250,584.500000,1.046429,43.750000,39.142857,82.928571,10.642857,44.785714,33.892857,49.821429,70.535714,12.550000,1.828571,248.964286,78.678571,0.0
4,2013-01-05,0.755489,0.444644,585.311111,1.304444,54.355556,45.333333,99.622222,17.822222,42.800000,30.911111,18.688889,62.755556,14.053333,1.420000,181.511111,73.866667,0.0


In [722]:
data_day_avg.to_csv(str("/Users/danielbustillos/Documents/servicio/Contaminación PM10/Outputs/por_dia/cont_dia" + a + ".csv"))